In [167]:
import sympy as sp
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

# Домашнее задание

Математический анализ
69 баллов

**Задача 1** (14 баллов)

Найдите экстремумы функции:

$$
f(x, y, z)=2 x^3+2 x y+2 x z+y^2+z^2+2 y-8 .
$$

Распишите подробное решение.

In [168]:
# Определяем символьные переменные
x, y, z = sp.symbols('x y z')

# Определяем функцию
f = 2*x**3 + 2*x*y + 2*x*z + y**2 + z**2 + 2*y - 8

# Находим частные производные первого порядка
fx = sp.diff(f, x)
fy = sp.diff(f, y)
fz = sp.diff(f, z)
print(f"""Частные производные первого порядка:
по x: {fx},
по y: {fy},
по z: {fz}""")

# Решаем систему уравнений
results = sp.solve((fx, fy, fz), (x, y, z))
print(f'Стационарные точки:')
print(*results)

# Определяем матрицу Гессе производными второго порядка
hesse = sp.Matrix([
    [sp.diff(f, x, x), sp.diff(f, x, y), sp.diff(f, x, z)],
    [sp.diff(f, y, x), sp.diff(f, y, y), sp.diff(f, y, z)],
    [sp.diff(f, z, x), sp.diff(f, z, y), sp.diff(f, z, z)]])

print('Экстремумы:')
# Проверяем стационарные точки, на основании собственных чисел и знака определителя определяем тип экстремума.
for res in results:
    H = hesse.subs({x: res[0], y: res[1], z: res[2]})
    eigenval = H.eigenvals()
    if all(e > 0 for e in eigenval):
        print(f'{res} - точка локального минимума')
    elif all(e < 0 for e in eigenval):
        print(f'{res} - точка локального максимума')
    else:
        print(f'{res} - седловая точка')

Частные производные первого порядка:
по x: 6*x**2 + 2*y + 2*z,
по y: 2*x + 2*y + 2,
по z: 2*x + 2*z
Стационарные точки:
(-1/3, -2/3, 1/3) (1, -2, -1)
Экстремумы:
(-1/3, -2/3, 1/3) - седловая точка
(1, -2, -1) - точка локального минимума


**Задача 2** (25 баллов)

Найдите условные экстремумы функции:

$$
f(x, y)=4 x+8 y, y^2-2 x y+5=0
$$

Вычислите результат самостоятельно (вручную) и с помощью Python. Сравните результаты.

Формулируем метод множителей Лагранжа.  
Пусть $ L(x,y,λ)=f(x,y)+λg(x,y) $, тогда:  
$ \mathcal{L}(x, y, \lambda) = 4x + 8y + \lambda(y^2 - 2xy + 5) $  

Находим частные производные функции Лагранжа по $x$, $y$ и $λ$ и приравниваем их к нулю:  
$ \frac{\partial \mathcal{L}}{\partial x} = 4 - 2\lambda y = 0 $  
$ \frac{\partial \mathcal{L}}{\partial y} = 8 + 2\lambda y - 2\lambda x = 0 $  
$ \frac{\partial \mathcal{L}}{\partial \lambda} = y^2 - 2xy + 5 = 0 $  

Из первого уравнения выражаем $λ$:  
$ \lambda = \frac{4}{2y} = \frac{2}{y} $

Подставляем $λ$ во второе уравнение:  
$ 8 + 2\left(\frac{2}{y}\right)y - 2\left(\frac{2}{y}\right)x = 0 $

Упрощаем:  
$ 8 + 4 - \frac{4x}{y} = 0 $  
$ 12 = \frac{4x}{y} $

Находим x:  
$ x = 3y $

Подставляем $x=3y$ в третье уравнение:  
$ y^2 - 2(3y)y + 5 = 0 $  
$ y^2 - 6y^2 + 5 = 0 $  
$ -5y^2 + 5 = 0 $  
$ y^2 = 1 $  

Отсюда $y=±1$.  
Соответственно, $x=3y=±3$.

Точки $(3,1)$ и $(−3,−1)$ являются условными экстремумами функции.

In [169]:
# Определяем переменные
x, y, lam = sp.symbols('x y lambda')

# Определяем функции
f = 4*x + 8*y
g = y**2 - 2*x*y + 5

# Определяем функцию Лагранжа
L = f + lam*g

# Находим частные производные
dL_dx = sp.diff(L, x)
dL_dy = sp.diff(L, y)
dL_dlam = sp.diff(L, lam)

# Решаем систему уравнений
results = sp.solve([dL_dx, dL_dy, dL_dlam], (x, y, lam))

print((results[0][0], results[0][1]), (results[1][0], results[1][1]))
print("""Условные экстремумы функции совпадают, как при ручном решении, так и при решении средствами sympy.""")

(-3, -1) (3, 1)
Условные экстремумы функции совпадают, как при ручном решении, так и при решении средствами sympy.


**Задача 3** (30 баллов)

Вам предложены данные с информацией о успеваемости студентов:

Независимые переменные:
* Hours Studied: Общее количество часов, потраченных на учебу каждым студентом.
* Previous Scores: Баллы, полученные студентами на предыдущих экзаменах.
* Sleep Hours: Среднее количество часов сна студента в сутки.
* Sample Question Papers Practiced: Количество пробных экзаменационных работ, с которыми студент занимался.
  
Целевая переменная:
* Performance Index: Показатель общей успеваемости каждого студента. Индекс успеваемости отражает академическую успеваемость студента и округляется до ближайшего целого числа. Индекс варьируется от 10 до 100, при этом более высокие значения свидетельствуют о более высокой успеваемости.

**Решите задачу линейной регрессии, реализовав градиентный спуск самостоятельно, не используя готовое решение из какой-либо библиотеки.**

In [170]:
df = pd.read_csv('./Student_Performance.csv')
df.drop('Extracurricular Activities', axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hours Studied                     10000 non-null  int64  
 1   Previous Scores                   10000 non-null  int64  
 2   Sleep Hours                       10000 non-null  int64  
 3   Sample Question Papers Practiced  10000 non-null  int64  
 4   Performance Index                 10000 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 390.8 KB


**Ручная реализация:**

In [171]:
from sklearn.preprocessing import StandardScaler

# Подготавливаем данные
X = df[['Hours Studied', 'Previous Scores', 'Sleep Hours', 'Sample Question Papers Practiced']].values
y = df['Performance Index'].values.reshape(-1, 1)

# Нормализуем данные
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

# Добавляем столбец с константой для смещения
X_norm = np.append(np.ones((len(df),1)), X_norm, axis = 1)

def compute_cost(w, X, y):
    """
    Функция вычисляет ошибку(потери) модели, которая определяется как сумма квадратов разностей между предсказанными значениями 
    и целевой переменной.
    """
    n = len(y)
    pred = X.dot(w)
    err = np.square(pred - y)
    cost = (1 / (2 * n)) * np.sum(err)
    
    return cost

def gradient_descent(X, y, w, alpha, num_iters):
    """
    Функция реализует процесс градиентного спуска. 
    Она обновляет параметры модели на основе градиентов, чтобы минимизировать ошибку(потери).
    """
    n = len(y)
    costs = []
    for i in range(num_iters):
        pred = X.dot(w)
        err = pred - y
        # Вычисляем градиент
        gradients = (1 / n) * X.T.dot(err)
        # Обновляем параметры
        w -= alpha * gradients
        # Сохраняем потери на каждой итерации
        costs.append(compute_cost(w, X, y))
    
    return w, costs

# Задаем начальный вектор коэффициентов и гиперпараметры
w = np.zeros((X_norm.shape[1], 1))
alpha = 0.05
num_iters = 10000

# Вычисляем коэффициенты модели
w, costs = gradient_descent(X_norm, y, w, alpha, num_iters)

**Реализация с помощью sklearn:**

In [172]:
# Создаём модель линейной регрессии
model = LinearRegression()

# Вычисляем коэффициенты регрессии
model.fit(X_norm, y)

LinearRegression()

In [173]:
from sklearn.metrics import mean_absolute_percentage_error

# Сравниваем ручную реализацию и модель из библиотеки sklearn
new_y = np.dot(X_norm, w)
new_y_skl = model.predict(X_norm)

mape = mean_absolute_percentage_error(y, new_y)*100
mape_skl = mean_absolute_percentage_error(y, new_y_skl)*100

manual = [float(i[0]) for i in w]
skl = [float(i) for i in np.hstack([model.intercept_, model.coef_[0][1:]])]

print('Сравнение параметров:')
print(f'Ручная реализация: \t{np.round(manual, 5)}')
print(f'Реализация sklearn: \t{np.round(skl, 5)}')
print()

print('Сравнение метрики MAPE:')
print(f'Ручная реализация: \t{mape:.2f}%')
print(f'Реализация sklearn: \t{mape_skl:.2f}%')
print()

print("""Сравнение показывает, что при точности до 5 знаков после точки, коэффициенты ручной и библиотечной реализаций полностью совпадают.
Средняя абсолютная процентная ошибка (MAPE) равна 3.50% для обеих реализаций, что помимо идентичности результатов, говорит о хорошем
качестве моделей.""")

Сравнение параметров:
Ручная реализация: 	[55.2248   7.38804 17.66457  0.80776  0.55967]
Реализация sklearn: 	[55.2248   7.38804 17.66457  0.80776  0.55967]

Сравнение метрики MAPE:
Ручная реализация: 	3.50%
Реализация sklearn: 	3.50%

Сравнение показывает, что при точности до 5 знаков после точки, коэффициенты ручной и библиотечной реализаций полностью совпадают.
Средняя абсолютная процентная ошибка (MAPE) равна 3.50% для обеих реализаций, что помимо идентичности результатов, говорит о хорошем
качестве моделей.
